<a href="https://colab.research.google.com/github/prometheusDE/mylastai/blob/master/mylastai_0008_project_handlevideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

Using TensorFlow backend.


In [22]:
import os.path
from os import path

from google.colab import files

count = 0
videoFile = "/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001.MP4"
print ("File exists:"+str(path.exists(videoFile)))
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
print(cap)
frameRate = cap.get(5) #frame rate
print(frameRate)
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame%05d.png" % count;count+=1
        retval = cv2.imwrite(filename, frame)
        print(filename)
        print(retval)
cap.release()
print ("Done!")

File exists:True
<VideoCapture 0x7f12eeaffdb0>
29.97002997002997
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00000.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00001.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00002.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00003.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00004.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00005.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00006.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00007.png
True
/content/gdrive/My Drive/CycleAI/20200309/2020-03-09_CycleAI_0002_Hinfahrt_IFF_001_frame00008.png
True
/content

KeyboardInterrupt: ignored